In [1]:
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd
from pandas import Index

In [2]:
scope = ["https://spreadsheets.google.com/feeds", 
"https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file("../../exampleetl-437815-e521d43e4b2e.json", scopes=scope)

In [3]:
client = gspread.authorize(creds)
sheet = client.open("e-commerce_test").sheet1 

In [4]:
data = sheet.get_all_records(expected_headers=['date', 'city', 'product', 'unit_sales', 'unit_price'])
df_raw = pd.DataFrame(data)

In [5]:
df_raw.head()

,date,city,product,unit_sales,unit_price
0,2024-02-01,Aveiro,mobile_phone,8,"$1,030.81"
1,2024-02-26,Sintra,mobile_phone,10,"$1,016.33"
2,2024-01-09,Porto,computer,3,$516.55
3,2024-02-14,Braga,computer,9,$612.37
4,2024-01-02,Porto,computer,10,"$1,055.75"


In [6]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        11 non-null     object
 1   city        11 non-null     object
 2   product     11 non-null     object
 3   unit_sales  11 non-null     int64 
 4   unit_price  11 non-null     object
dtypes: int64(1), object(4)
memory usage: 572.0+ bytes


In [14]:
valid_product_categories = ['computer', 'mobile_phone']
valid_city_name = []

df_clean = (
    df_raw
    .assign(unit_price=lambda _df: _df.unit_price.str.replace('$','').str.replace(',','').astype(float)) 
    .assign(date=lambda _df: pd.to_datetime(_df.date))
    .assign(product=lambda _df: pd.Categorical(_df['product'], categories=valid_product_categories))
)

In [15]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        11 non-null     datetime64[ns]
 1   city        11 non-null     object        
 2   product     11 non-null     category      
 3   unit_sales  11 non-null     int64         
 4   unit_price  11 non-null     float64       
dtypes: category(1), datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 619.0+ bytes
